In [1]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

Using TensorFlow backend.


In [22]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(119,activation='softmax')(x) #final layer with softmax activation

In [23]:
model=Model(inputs=base_model.input,outputs=preds)

In [24]:
for i,layer in enumerate(model.layers):
    print(i,layer.name)

0 input_2
1 conv1_pad
2 conv1
3 conv1_bn
4 conv1_relu
5 conv_dw_1
6 conv_dw_1_bn
7 conv_dw_1_relu
8 conv_pw_1
9 conv_pw_1_bn
10 conv_pw_1_relu
11 conv_pad_2
12 conv_dw_2
13 conv_dw_2_bn
14 conv_dw_2_relu
15 conv_pw_2
16 conv_pw_2_bn
17 conv_pw_2_relu
18 conv_dw_3
19 conv_dw_3_bn
20 conv_dw_3_relu
21 conv_pw_3
22 conv_pw_3_bn
23 conv_pw_3_relu
24 conv_pad_4
25 conv_dw_4
26 conv_dw_4_bn
27 conv_dw_4_relu
28 conv_pw_4
29 conv_pw_4_bn
30 conv_pw_4_relu
31 conv_dw_5
32 conv_dw_5_bn
33 conv_dw_5_relu
34 conv_pw_5
35 conv_pw_5_bn
36 conv_pw_5_relu
37 conv_pad_6
38 conv_dw_6
39 conv_dw_6_bn
40 conv_dw_6_relu
41 conv_pw_6
42 conv_pw_6_bn
43 conv_pw_6_relu
44 conv_dw_7
45 conv_dw_7_bn
46 conv_dw_7_relu
47 conv_pw_7
48 conv_pw_7_bn
49 conv_pw_7_relu
50 conv_dw_8
51 conv_dw_8_bn
52 conv_dw_8_relu
53 conv_pw_8
54 conv_pw_8_bn
55 conv_pw_8_relu
56 conv_dw_9
57 conv_dw_9_bn
58 conv_dw_9_relu
59 conv_pw_9
60 conv_pw_9_bn
61 conv_pw_9_relu
62 conv_dw_10
63 conv_dw_10_bn
64 conv_dw_10_relu
65 conv_pw_10

In [25]:
for layer in model.layers:
    layer.trainable=False
# or if we want to set the first 20 layers of the network to be non-trainable
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [21]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('Fruit-Images-Dataset/Training/',
                                                 #target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 50403 images belonging to 119 classes.


In [7]:
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [26]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
food_model = model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=5)

Epoch 1/5
1575/1575 [==============================] - 2535s 2s/step - loss: 0.4128 - acc: 0.8950
Epoch 2/5
1575/1575 [==============================] - 2492s 2s/step - loss: 0.1730 - acc: 0.9574
Epoch 3/5
1575/1575 [==============================] - 2378s 2s/step - loss: 0.1433 - acc: 0.9647
Epoch 4/5
1575/1575 [==============================] - 986s 626ms/step - loss: 0.1306 - acc: 0.9698
Epoch 5/5
1575/1575 [==============================] - 987s 627ms/step - loss: 0.1031 - acc: 0.9763


In [12]:
model.save("food_v1.h5")

In [28]:
model.save("all_groceries_model.h5")

In [29]:
# test_folder

test_generator=train_datagen.flow_from_directory('Fruit-Images-Dataset/Test/',
                                                 #target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 16727 images belonging to 104 classes.


In [42]:
#step_size_test=test_generator.n//test_generator.batch_size
filenames = test_generator.filenames[:20]
nb_samples = len(filenames)

predict = model.predict_generator(test_generator,steps = nb_samples)
predicted_class_indices=np.argmax(predict,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [40]:
labels

{0: 'Apple Braeburn',
 1: 'Apple Golden 1',
 2: 'Apple Golden 2',
 3: 'Apple Golden 3',
 4: 'Apple Granny Smith',
 5: 'Apple Red 1',
 6: 'Apple Red 2',
 7: 'Apple Red 3',
 8: 'Apple Red Delicious',
 9: 'Apple Red Yellow 1',
 10: 'Apple Red Yellow 2',
 11: 'Apricot',
 12: 'Asparagus',
 13: 'Aubergine',
 14: 'Avocado',
 15: 'Avocado ripe',
 16: 'Banana',
 17: 'Banana Lady Finger',
 18: 'Banana Red',
 19: 'Brown-Cap-Mushroom',
 20: 'Cabbage',
 21: 'Cactus fruit',
 22: 'Cantaloupe 1',
 23: 'Cantaloupe 2',
 24: 'Carambula',
 25: 'Carrots',
 26: 'Cherry 1',
 27: 'Cherry 2',
 28: 'Cherry Rainier',
 29: 'Cherry Wax Black',
 30: 'Cherry Wax Red',
 31: 'Cherry Wax Yellow',
 32: 'Chestnut',
 33: 'Clementine',
 34: 'Cocos',
 35: 'Cucumber',
 36: 'Dates',
 37: 'Garlic',
 38: 'Ginger',
 39: 'Granadilla',
 40: 'Grape Blue',
 41: 'Grape Pink',
 42: 'Grape White',
 43: 'Grape White 2',
 44: 'Grape White 3',
 45: 'Grape White 4',
 46: 'Grapefruit Pink',
 47: 'Grapefruit White',
 48: 'Guava',
 49: 'Hazel

In [75]:
from tkinter import filedialog
from tkinter.filedialog import askopenfilename
test_datagen = ImageDataGenerator(rescale=1. / 255)
#directory = filedialog.askdirectory()
#file_list = os.listdir(directory)
#pred_dir = 'RecipeCoach/scripts/ToPredict/'

pred_dir = filedialog.askdirectory()

photo_generator=test_datagen.flow_from_directory(directory=pred_dir,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)



Found 23 images belonging to 1 classes.


In [76]:
photo_generator.reset()

In [77]:
pred=model.predict_generator(photo_generator,verbose=1,steps=1)
predicted_class_indices=np.argmax(pred,axis=1)

1/1 [==============================] - 0s 435ms/step


In [78]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

In [79]:
labels

{0: 'Apple Braeburn',
 1: 'Apple Golden 1',
 2: 'Apple Golden 2',
 3: 'Apple Golden 3',
 4: 'Apple Granny Smith',
 5: 'Apple Red 1',
 6: 'Apple Red 2',
 7: 'Apple Red 3',
 8: 'Apple Red Delicious',
 9: 'Apple Red Yellow 1',
 10: 'Apple Red Yellow 2',
 11: 'Apricot',
 12: 'Asparagus',
 13: 'Aubergine',
 14: 'Avocado',
 15: 'Avocado ripe',
 16: 'Banana',
 17: 'Banana Lady Finger',
 18: 'Banana Red',
 19: 'Brown-Cap-Mushroom',
 20: 'Cabbage',
 21: 'Cactus fruit',
 22: 'Cantaloupe 1',
 23: 'Cantaloupe 2',
 24: 'Carambula',
 25: 'Carrots',
 26: 'Cherry 1',
 27: 'Cherry 2',
 28: 'Cherry Rainier',
 29: 'Cherry Wax Black',
 30: 'Cherry Wax Red',
 31: 'Cherry Wax Yellow',
 32: 'Chestnut',
 33: 'Clementine',
 34: 'Cocos',
 35: 'Cucumber',
 36: 'Dates',
 37: 'Garlic',
 38: 'Ginger',
 39: 'Granadilla',
 40: 'Grape Blue',
 41: 'Grape Pink',
 42: 'Grape White',
 43: 'Grape White 2',
 44: 'Grape White 3',
 45: 'Grape White 4',
 46: 'Grapefruit Pink',
 47: 'Grapefruit White',
 48: 'Guava',
 49: 'Hazel

In [80]:
predictions = [labels[k] for k in predicted_class_indices]

In [81]:
predictions

['Cantaloupe 1',
 'Red-Beet',
 'Peach 2',
 'Carrots',
 'Cherry Wax Yellow',
 'Cantaloupe 2',
 'Physalis with Husk',
 'Cherry Wax Yellow',
 'Cantaloupe 2',
 'Tomato 4',
 'Physalis with Husk',
 'Tomato Maroon',
 'Apple Golden 2',
 'Onion',
 'Asparagus',
 'Physalis with Husk',
 'Cantaloupe 2',
 'Red-Beet',
 'Asparagus',
 'Juice',
 'Physalis with Husk',
 'Red-Beet',
 'Pineapple']

In [ ]:
#filenames = photo_generator.filenames
#nb_samples = len(filenames)

#predict = model.predict_generator(photo_generator,steps = nb_samples)

#predict = model.predict(file_list)
#predicted_class_indices=np.argmax(predict,axis=1)
#predictions = [labels[k] for k in predicted_class_indices]
#print(predictions)